# Coordinate Reference System (CRS) Management

Rasterix uses [XProj](https://xproj.readthedocs.io/en/latest/) to store and manage the CRS of a dataset

In [16]:
import rasterix
import xarray as xr
import xproj

# GDAL settings for remote data
import os
os.environ['VSICURL_PC_URL_SIGNING']='YES'
os.environ['GDAL_DISABLE_READDIR_ON_OPEN'] = 'EMPTY_DIR'

xr.set_options(display_expand_indexes=True);

In [14]:
url = "https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/2024/01_Jan/S_20240101_concentration_v3.0.tif"
da = xr.open_dataarray(url, engine="rasterio")

# How to get this to automatically pick up CRS that is detected by rioxarray?
# Can this be done automatically by assign_index?
# Should assign_index happen automatically? or would this be done with engine='rasterix')
da = rasterix.assign_index(da).proj.assign_crs(crs=da.rio.crs)
da

<xarray.DataArray 'band_data' (band: 1, y: 332, x: 316)> Size: 420kB
[104912 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B ...
  * x            (x) float64 3kB -3.938e+06 -3.912e+06 ... 3.912e+06 3.938e+06
  * y            (y) float64 3kB 4.338e+06 4.312e+06 ... -3.912e+06 -3.938e+06
  * crs          int64 8B 0
Indexes:
  ┌ x        RasterIndex
  └ y
    crs      CRSIndex (crs=EPSG:3412)
Attributes:
    AREA_OR_POINT:  Area

In [23]:
# Make sure we can save this back to a geotiff 
# Hmmm, this warning isn't clear to me
da.rio.to_raster('/tmp/S_20240101_concentration_v3.0.tif')

/home/jovyan/.local/share/hatch/env/virtual/rasterix/UNYVAJKM/test.py3.13/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3153: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  result = runner(coro)


In [27]:
!wget -nc https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/2024/01_Jan/S_20240101_concentration_v3.0.tif
!ls -l S_20240101_concentration_v3.0.tif

File ‘S_20240101_concentration_v3.0.tif’ already there; not retrieving.

-rw-r--r-- 1 jovyan jovyan 606187 Jan  2  2024 S_20240101_concentration_v3.0.tif


In [28]:
# Are the tiffs the same?
!ls -l /tmp/S_20240101_concentration_v3.0.tif

-rw-r--r-- 1 jovyan jovyan 213021 May 13 13:51 /tmp/S_20240101_concentration_v3.0.tif


## Two rasters with same GeoTransform but different CRS

Here is a case for needing to account for both affine and CRS. [Sentinel-2](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a) is delivered on the MGRS grid, which divides UTM Zones into smaller units. Adjacent zones have the same affines, and therefore appear to have the same coordinates. But the CRS differentiates them.

We want Xarray to raise an error in this case before attempting any computations.

In [22]:
def load_raster(href, overview_level=3, masked=True):
    da = xr.open_dataarray(href, 
                           engine='rasterio', 
                           masked=masked,
                           open_kwargs=dict(overview_level=overview_level),
                           chunks='auto').squeeze()
    
    return rasterix.assign_index(da).proj.assign_crs(crs=da.rio.crs)


url = 'https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/10/T/DS/2025/04/14/S2C_MSIL2A_20250414T190931_N0511_R056_T10TDS_20250415T001314.SAFE/GRANULE/L2A_T10TDS_A003172_20250414T191846/IMG_DATA/R10m/T10TDS_20250414T190931_B04_10m.tif'
da1 = load_raster(url)

url = 'https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/11/T/MM/2025/04/13/S2B_MSIL2A_20250413T184919_N0511_R113_T11TMM_20250413T224733.SAFE/GRANULE/L2A_T11TMM_A042324_20250413T185100/IMG_DATA/R10m/T11TMM_20250413T184919_B04_10m.tif'
da2 = load_raster(url)
    
da1 + da2 

MergeError: conflicting values/indexes on objects to be combined for coordinate 'crs'
first index: CRSIndex
<Projected CRS: EPSG:32610>
Name: WGS 84 / UTM zone 10N
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: UTM zone 10N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

second index: CRSIndex
<Projected CRS: EPSG:32611>
Name: WGS 84 / UTM zone 11N
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: UTM zone 11N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

first variable: <xarray.Variable ()> Size: 8B
array(0)
second variable: <xarray.Variable ()> Size: 8B
array(0)
